1. setup

Importera in bibliotek
Sätt k-värde 
Sät top_n-värde

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_absolute_error

Nedan lägger vi till csv-filerna med Användares betyg och spelen 
....

In [2]:
df_ratings = pd.read_csv("dataset/user_ratings.csv")
df_games = pd.read_csv("dataset/games.csv")
df_ratings.info()
#df_games.info()
df = pd.merge(df_ratings, df_games[['BGGId', 'Name']], on='BGGId', how='left')
df.info()   


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18942215 entries, 0 to 18942214
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 433.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18942215 entries, 0 to 18942214
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   BGGId     int64  
 1   Rating    float64
 2   Username  object 
 3   Name      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 722.6+ MB


In [3]:
df_ratings

,BGGId,Rating,Username
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP
...,...,...,...
18942210,165521,3.0,rseater
18942211,165521,3.0,Bluefox86
18942212,165521,3.0,serginator
18942213,193488,1.0,CaptainCattan


In [4]:
df.describe()

,BGGId,Rating
count,1.894222e+07,1.894222e+07
mean,1.099533e+05,7.130957e+00
std,9.280217e+04,1.543023e+00
min,1.000000e+00,1.000000e-04
25%,1.611000e+04,6.000000e+00
50%,1.075290e+05,7.000000e+00
75%,1.813040e+05,8.000000e+00
max,3.491610e+05,1.000000e+01


2 Preprocessing

Handle missing valuee

In [5]:
any_null_values = df.isnull().any()
any_null_values

null_counts = df.isnull().sum()
print(null_counts)

BGGId        0
Rating       0
Username    63
Name         0
dtype: int64


In [6]:
df = df.dropna()

In [7]:
df.isnull().any()

BGGId       False
Rating      False
Username    False
Name        False
dtype: bool

In [8]:
df

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942210,165521,3.0,rseater,Rory's Story Cubes: Medic
18942211,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942212,165521,3.0,serginator,Rory's Story Cubes: Medic
18942213,193488,1.0,CaptainCattan,Airlines


In [9]:
duplicates = df.duplicated().value_counts()
duplicates

False    18942152
dtype: int64

In [10]:
duplicates.describe()

count           1.0
mean     18942152.0
std             NaN
min      18942152.0
25%      18942152.0
50%      18942152.0
75%      18942152.0
max      18942152.0
dtype: float64

Remove duplicate ratings for the same game by the same user (due to different version of the game etc)

In [11]:
# Calculate the mean rating for each group of duplicate rows
mean_ratings = df.groupby(['Username', 'BGGId'])['Rating'].mean()

In [12]:
mean_ratings

Username        BGGId 
 Fu_Koios       112092    9.0
                223033    9.0
 beastvol       13        8.0
                118       7.0
                278       7.0
                         ... 
Æleksandr Þræð  2399      7.0
                2932      6.0
                5451      8.0
                5554      7.0
                9962      8.0
Name: Rating, Length: 18909465, dtype: float64

In [13]:
# Filter the DataFrame for the specific user and item
specific_user_item_ratings = df[(df['Username'] == 'hariseldoncwb') & (df['BGGId'] == 135382)]

# Calculate the mean rating for the specific user and item
mean_rating_specific_user_item = specific_user_item_ratings['Rating'].mean()
mean_rating_specific_user_item

5.5

In [14]:
# Merge the mean ratings back to the original DataFrame based on 'Username' and 'Name'
df_removed_duplicates = df.merge(mean_ratings, on=['Username', 'BGGId'], suffixes=('', '_mean'))

In [15]:
df_removed_duplicates

,BGGId,Rating,Username,Name,Rating_mean
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,8.0
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition,8.0
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition,8.0
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition,8.0
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition,8.0
...,...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic,3.0
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic,3.0
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic,3.0
18942150,193488,1.0,CaptainCattan,Airlines,1.0


In [16]:
# Filter the DataFrame to keep only the original rows (non-duplicates) and the calculated mean ratings
df_unique = df_removed_duplicates[~df_removed_duplicates.duplicated(subset=['Username', 'BGGId'], keep=False)]

In [17]:
df_unique

,BGGId,Rating,Username,Name,Rating_mean
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,8.0
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition,8.0
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition,8.0
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition,8.0
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition,8.0
...,...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic,3.0
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic,3.0
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic,3.0
18942150,193488,1.0,CaptainCattan,Airlines,1.0


In [18]:
# Drop the original 'Rating' column and rename the mean rating column
df_dropped = df_unique.drop(columns=['Rating']).rename(columns={'Rating_mean': 'Rating'})

In [19]:
# Reindex the columns to move 'Rating' to the second position
column_order = ['BGGId', 'Rating', 'Username', 'Name']
df_dropped = df_dropped.reindex(columns=column_order)

In [20]:
df_dropped

,BGGId,Rating,Username,Name
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition
1,213788,8.0,tachyon14k,Ascension: Year Three Collector's Edition
2,213788,8.0,Ungotter,Ascension: Year Three Collector's Edition
3,213788,8.0,brainlocki3,Ascension: Year Three Collector's Edition
4,213788,8.0,PPMP,Ascension: Year Three Collector's Edition
...,...,...,...,...
18942147,165521,3.0,rseater,Rory's Story Cubes: Medic
18942148,165521,3.0,Bluefox86,Rory's Story Cubes: Medic
18942149,165521,3.0,serginator,Rory's Story Cubes: Medic
18942150,193488,1.0,CaptainCattan,Airlines


In [21]:
# Step 1: Group by 'Username' and calculate the average rating for each user
user_avg_ratings = df_dropped.groupby('Username')['Rating'].mean().reset_index()


Filter dataset

In [22]:
# Step 2: Merge the average ratings back into the original DataFrame
df_adjusted_mean = pd.merge(df_dropped, user_avg_ratings, on='Username', suffixes=('', '_avg'))


In [23]:
df_adjusted_mean

,BGGId,Rating,Username,Name,Rating_avg
0,213788,8.0,Tonydorrf,Ascension: Year Three Collector's Edition,7.367589
1,206593,8.0,Tonydorrf,Stronghold: Undead (Second Edition) – Kickstar...,7.367589
2,267333,7.0,Tonydorrf,Goetia: Nine Kings of Solomon,7.367589
3,249824,6.0,Tonydorrf,Munchkin Harry Potter Deluxe,7.367589
4,301085,6.0,Tonydorrf,Rebis,7.367589
...,...,...,...,...,...
18878027,195742,5.0,codyk4545,The Godfather: A New Don,5.000000
18878028,215065,2.0,UnrealTravis,75 Gnom' Street,2.000000
18878029,127096,3.0,Elianto1989,Metal Adventures,3.000000
18878030,126471,2.0,DaLanimilbus,Nowhere to Go,2.000000


In [24]:

# Step 3: Subtract the average rating from each rating to get the adjusted rating
df_adjusted_mean['Adjusted_Rating'] = df_adjusted_mean['Rating'] - df_adjusted_mean['Rating_avg']

column_order = ['BGGId', 'Rating', 'Rating_avg', 'Adjusted_Rating', 'Username', 'Name']
df_adjusted_mean = df_adjusted_mean.reindex(columns=column_order)
# Now df contains the adjusted ratings in the 'Adjusted_Rating' column

In [25]:
df_adjusted_mean

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
0,213788,8.0,7.367589,0.632411,Tonydorrf,Ascension: Year Three Collector's Edition
1,206593,8.0,7.367589,0.632411,Tonydorrf,Stronghold: Undead (Second Edition) – Kickstar...
2,267333,7.0,7.367589,-0.367589,Tonydorrf,Goetia: Nine Kings of Solomon
3,249824,6.0,7.367589,-1.367589,Tonydorrf,Munchkin Harry Potter Deluxe
4,301085,6.0,7.367589,-1.367589,Tonydorrf,Rebis
...,...,...,...,...,...,...
18878027,195742,5.0,5.000000,0.000000,codyk4545,The Godfather: A New Don
18878028,215065,2.0,2.000000,0.000000,UnrealTravis,75 Gnom' Street
18878029,127096,3.0,3.000000,0.000000,Elianto1989,Metal Adventures
18878030,126471,2.0,2.000000,0.000000,DaLanimilbus,Nowhere to Go


In [26]:
# Group by 'Username' and count the number of rows for each group
grouped_user_counts = df_adjusted_mean.groupby('Username').size()
grouped_user_counts

Username
 Fu_Koios              2
 beastvol              9
 mycroft              14
 woh                   5
(mostly) harmless      1
                    ... 
zzzuzu                39
zzzvone               21
zzzxxxyyy             32
zzzzzane             154
Æleksandr Þræð        12
Length: 411312, dtype: int64

In [27]:
# Group by 'BGGId' and count the number of rows for each group
grouped_counts = df_adjusted_mean.groupby('BGGId').size()
grouped_counts

BGGId
1          5295
2           560
3         15077
4           342
5         18152
          ...  
347146       42
347521       71
348955       67
349131       66
349161        7
Length: 21925, dtype: int64

In [28]:
game_ratings = 3000

rating_counter = df_adjusted_mean['BGGId'].value_counts()

filtered_out = rating_counter[rating_counter < game_ratings].index

games_mask = df_adjusted_mean['BGGId'].isin(filtered_out)
#games_mask = df['BGGId'].isin(rating_counter[rating_counter < game_ratings].index)
#games_mask
df_adjusted_mean.drop(index=df_adjusted_mean[games_mask].index, inplace=True)

df_adjusted_mean

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
44,36218,7.0,7.367589,-0.367589,Tonydorrf,Dominion
45,133473,6.0,7.367589,-1.367589,Tonydorrf,Sushi Go!
46,167791,9.0,7.367589,1.632411,Tonydorrf,Terraforming Mars
47,230802,8.0,7.367589,0.632411,Tonydorrf,Azul
48,174430,9.0,7.367589,1.632411,Tonydorrf,Gloomhaven
...,...,...,...,...,...,...
18852015,125548,10.0,10.000000,0.000000,brocespringsteen,Pixel Tactics
18852016,125548,10.0,10.000000,0.000000,Czerstwy,Pixel Tactics
18852018,125548,10.0,10.000000,0.000000,Shining Creations,Pixel Tactics
18852019,125548,10.0,10.000000,0.000000,MissTaluva,Pixel Tactics


In [29]:
user_ratings = 100

rating_counter = df_adjusted_mean['Username'].value_counts()

filtered_out = rating_counter[rating_counter < user_ratings].index

user_mask = df_adjusted_mean['Username'].isin(filtered_out)

df_adjusted_mean.drop(index=df_adjusted_mean[user_mask].index, inplace=True)

df_adjusted_mean

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
386,178900,7.0,7.641304,-0.641304,PPMP,Codenames
387,14996,7.0,7.641304,-0.641304,PPMP,Ticket to Ride: Europe
388,40692,7.0,7.641304,-0.641304,PPMP,Small World
389,822,8.0,7.641304,0.358696,PPMP,Carcassonne
390,70323,7.0,7.641304,-0.641304,PPMP,King of Tokyo
...,...,...,...,...,...,...
18691910,634,10.0,10.000000,0.000000,AbsterPro,River Dragons
18691911,174973,10.0,10.000000,0.000000,AbsterPro,Boss Monster 2: The Next Level
18691912,191894,10.0,10.000000,0.000000,AbsterPro,Imagine
18691913,19526,10.0,10.000000,0.000000,AbsterPro,Elasund: The First City


In [30]:
# Group by 'Username' and count the number of rows for each group
grouped_user_counts = df_adjusted_mean.groupby('Username').size()
grouped_user_counts

Username
-=Yod@=-     201
-Johnny-     245
-mIDE-       356
-toni-       137
...Hammer    123
            ... 
zzyzxuk      118
zzzabiss     296
zzzhsm       171
zzzoren      141
zzzzzane     136
Length: 31482, dtype: int64

In [31]:
# Group by 'BGGId' and count the number of rows for each group
grouped_counts = df_adjusted_mean.groupby('BGGId').size()
grouped_counts

BGGId
1         3647
3         8711
5         9065
7         1751
10        5111
          ... 
316554    5818
317985    3028
318977    2779
328871    1826
329465    1812
Length: 1167, dtype: int64

In [32]:
grouped_by_username = df_adjusted_mean.groupby('Username').size().sort_values(ascending=False)
grouped_by_username

Username
leffe dubbel            1081
Hessu68                 1020
oldgoat3769967          1002
TomVasel                 996
Pandorzecza              953
                        ... 
RoN_Rancor               100
generationX              100
Nathan G                 100
starvinmartian_games     100
chsbellboy               100
Length: 31482, dtype: int64

In [33]:
df_adjusted_mean

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
386,178900,7.0,7.641304,-0.641304,PPMP,Codenames
387,14996,7.0,7.641304,-0.641304,PPMP,Ticket to Ride: Europe
388,40692,7.0,7.641304,-0.641304,PPMP,Small World
389,822,8.0,7.641304,0.358696,PPMP,Carcassonne
390,70323,7.0,7.641304,-0.641304,PPMP,King of Tokyo
...,...,...,...,...,...,...
18691910,634,10.0,10.000000,0.000000,AbsterPro,River Dragons
18691911,174973,10.0,10.000000,0.000000,AbsterPro,Boss Monster 2: The Next Level
18691912,191894,10.0,10.000000,0.000000,AbsterPro,Imagine
18691913,19526,10.0,10.000000,0.000000,AbsterPro,Elasund: The First City


In [34]:
average_ratings = dict(zip(df_adjusted_mean['Username'], df_adjusted_mean['Rating_avg']))

Train test split

In [143]:
# Define a function to split the data for each user
def user_split(data, random_state, test_size=0.2):    
    train_to_concat = []
    test_to_concat = []
    for user, group in data.groupby('Username'):
        train_user, test_user = train_test_split(group, test_size=test_size, train_size=1-test_size, random_state=random_state)
        train_to_concat.append(train_user)
        test_to_concat.append(test_user)
    
    train = pd.concat(train_to_concat, ignore_index=True)
    test = pd.concat(test_to_concat, ignore_index=True)

    return train, test


In [153]:
# Split the data
train_set, test_set = user_split(df_adjusted_mean, 0, test_size=0.2)

In [150]:
train_set

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
0,90137,7.5,7.287829,0.212171,-=Yod@=-,Blood Bowl: Team Manager – The Card Game
1,9674,7.5,7.287829,0.212171,-=Yod@=-,Ingenious
2,239188,7.0,7.287829,-0.287829,-=Yod@=-,Chronicles of Crime
3,84876,8.0,7.287829,0.712171,-=Yod@=-,The Castles of Burgundy
4,68448,9.0,7.287829,1.712171,-=Yod@=-,7 Wonders
...,...,...,...,...,...,...
4754392,140934,9.0,7.393506,1.606494,zzzzzane,Arboretum
4754393,172308,6.5,7.393506,-0.893506,zzzzzane,Broom Service
4754394,95064,6.0,7.393506,-1.393506,zzzzzane,Ascension: Return of the Fallen
4754395,37111,9.0,7.393506,1.606494,zzzzzane,Battlestar Galactica: The Board Game


In [152]:
train_set

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
0,90137,7.5,7.287829,0.212171,-=Yod@=-,Blood Bowl: Team Manager – The Card Game
1,9674,7.5,7.287829,0.212171,-=Yod@=-,Ingenious
2,239188,7.0,7.287829,-0.287829,-=Yod@=-,Chronicles of Crime
3,84876,8.0,7.287829,0.712171,-=Yod@=-,The Castles of Burgundy
4,68448,9.0,7.287829,1.712171,-=Yod@=-,7 Wonders
...,...,...,...,...,...,...
4754392,140934,9.0,7.393506,1.606494,zzzzzane,Arboretum
4754393,172308,6.5,7.393506,-0.893506,zzzzzane,Broom Service
4754394,95064,6.0,7.393506,-1.393506,zzzzzane,Ascension: Return of the Fallen
4754395,37111,9.0,7.393506,1.606494,zzzzzane,Battlestar Galactica: The Board Game


In [146]:
test_set

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
0,39856,7.5,7.287829,0.212171,-=Yod@=-,Dixit
1,213460,8.0,7.287829,0.712171,-=Yod@=-,Unlock!: Escape Adventures
2,38054,6.0,7.287829,-1.287829,-=Yod@=-,Snow Tails
3,127023,8.0,7.287829,0.712171,-=Yod@=-,Kemet
4,2381,7.5,7.287829,0.212171,-=Yod@=-,Scattergories
...,...,...,...,...,...,...
1204355,37046,6.0,7.393506,-1.393506,zzzzzane,Ghost Stories
1204356,144733,8.5,7.393506,1.106494,zzzzzane,Russian Railroads
1204357,140620,6.5,7.393506,-0.893506,zzzzzane,Lewis & Clark: The Expedition
1204358,148228,7.0,7.393506,-0.393506,zzzzzane,Splendor


In [39]:
#Used to calculate MAE
actual_ratings_test_set = {}
for _, row in test_set.iterrows():
    username = row['Username']
    bggid = row['BGGId']
    rating = row['Rating']
    
    if username not in actual_ratings_test_set:
        actual_ratings_test_set[username] = {}
    
    actual_ratings_test_set[username][bggid] = rating

In [40]:
#Used in calculate predicted rating
actual_ratings_train_set = {}
for _, row in train_set.iterrows():
    username = row['Username']
    bggid = row['BGGId']
    rating = row['Rating']
    
    if username not in actual_ratings_train_set:
        actual_ratings_train_set[username] = {}
    
    actual_ratings_train_set[username][bggid] = rating

In [41]:
#Used to calculate MAE
rated_items_test_set = {}
for index, row in test_set.iterrows():
    username = row['Username']
    bggid = row['BGGId']
    
    if username not in rated_items_test_set:
        rated_items_test_set[username] = []
    
    rated_items_test_set[username].append(bggid)

In [42]:
#Used in calculate predicted rating
rated_items_train_set = {}
for index, row in train_set.iterrows():
    username = row['Username']
    bggid = row['BGGId']
    
    if username not in rated_items_train_set:
        rated_items_train_set[username] = []
    
    rated_items_train_set[username].append(bggid)

In [113]:
relevant_items_df = test_set.loc[test_set['Rating'] >= 7]
# Initialize an empty dictionary to store Username as keys and lists of BGGId values as values
relevant_items = {}

# Iterate over the filtered DataFrame to populate the dictionary
for username, bggid in relevant_items_df.groupby('Username')['BGGId']:
    # Convert the groupby object to a list of BGGId values
    bggid_list = list(bggid)
    # Store the list of BGGId values for the current Username in the dictionary
    relevant_items[username] = bggid_list

filtrera dataframe 

3 Similarity Calculation

 Skapa en matris av item-för-item-similarity baserat på cosine similarity

In [43]:
# Pivot the DataFrame so a user matrix, so rows represent users and columns represent items
user_item_matrix = train_set.pivot(index='Username', columns='BGGId', values='Rating').fillna(0)



In [44]:
user_item_matrix

BGGId,1,3,5,7,10,11,12,13,15,18,...,304420,306735,308765,312484,314040,316554,317985,318977,328871,329465
Username,,,,,,,,,,,,,,,,,,,,,
-=Yod@=-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
-Johnny-,0.0,4.0,7.0,0.0,4.0,0.0,5.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-mIDE-,0.0,8.0,0.0,0.0,0.0,7.5,0.0,7.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
-toni-,0.0,9.0,0.0,0.0,0.0,7.0,0.0,4.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...Hammer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzyzxuk,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
zzzabiss,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzzhsm,5.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
# Pivot the DataFrame so a user matrix, so rows represent users and columns represent items
user_item_matrix_with_nan = train_set.pivot(index='Username', columns='BGGId', values='Adjusted_Rating')

In [46]:
user_item_matrix_with_nan

BGGId,1,3,5,7,10,11,12,13,15,18,...,304420,306735,308765,312484,314040,316554,317985,318977,328871,329465
Username,,,,,,,,,,,,,,,,,,,,,
-=Yod@=-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.287829,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.287829,NaN,NaN
-Johnny-,NaN,-1.322245,1.677755,NaN,-1.322245,NaN,-0.322245,NaN,NaN,0.677755,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-mIDE-,NaN,0.964286,NaN,NaN,NaN,0.464286,NaN,-0.035714,NaN,0.964286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.964286,NaN,NaN
-toni-,NaN,2.552846,NaN,NaN,NaN,0.552846,NaN,-2.447154,NaN,2.552846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...Hammer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.347973,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzyzxuk,NaN,NaN,0.251701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.251701,NaN,NaN,NaN,NaN,NaN,NaN
zzzabiss,NaN,NaN,NaN,NaN,NaN,-0.828194,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zzzhsm,-1.461988,NaN,1.538012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# Used in recommend items

not_rated_items_train_set = {}
# Iterate over the rows of the DataFrame
for username, row in user_item_matrix_with_nan.iterrows():
    nan_bggid_list = []
    # Iterate over the columns (BGGIds)
    for bggid, value in row.items():
        # Check if the value is NaN
        if pd.isna(value):
            nan_bggid_list.append(bggid)
    # Store the list of BGGId values with NaN for the current Username in the dictionary
    not_rated_items_train_set[username] = nan_bggid_list


In [48]:
# Transpose the user-item matrix to obtain an item-user matrix
item_user_matrix = user_item_matrix.T

In [49]:
item_user_matrix

Username,-=Yod@=-,-Johnny-,-mIDE-,-toni-,...Hammer,.JcK.,0010,0b1_Ita,0ddjob,0hoHj,...,zzas,zzay,zzeroparticle,zztap,zzyzewitsch,zzyzxuk,zzzabiss,zzzhsm,zzzoren,zzzzzane
BGGId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,5.0,0.0,0.0
3,0.0,4.0,8.0,9.0,0.0,0.0,0.0,7.0,0.0,0.0,...,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.3,0.0,0.0,6.0,5.0,7.0,0.0,8.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316554,0.0,0.0,0.0,0.0,7.0,0.0,9.0,0.0,0.0,10.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
317985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
318977,7.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
item_user_matrix_adjusted = user_item_matrix_with_nan.T

In [51]:
item_user_matrix_adjusted

Username,-=Yod@=-,-Johnny-,-mIDE-,-toni-,...Hammer,.JcK.,0010,0b1_Ita,0ddjob,0hoHj,...,zzas,zzay,zzeroparticle,zztap,zzyzewitsch,zzyzxuk,zzzabiss,zzzhsm,zzzoren,zzzzzane
BGGId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,2.103896,NaN,NaN,NaN,2.17541,...,NaN,NaN,NaN,NaN,0.62141,NaN,NaN,-1.461988,NaN,NaN
3,NaN,-1.322245,0.964286,2.552846,NaN,NaN,NaN,-0.097356,NaN,NaN,...,0.453333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,1.677755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.253333,NaN,NaN,-0.832021,-1.37859,0.251701,NaN,1.538012,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,-1.322245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316554,NaN,NaN,NaN,NaN,0.347973,NaN,1.478261,NaN,NaN,2.17541,...,NaN,NaN,NaN,2.167979,NaN,NaN,NaN,NaN,NaN,NaN
317985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.17541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
318977,-0.287829,NaN,0.964286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
def calculate_adjusted_cosine_similarity(matrix_transposed):
    sparse_matrix = csr_matrix(matrix_transposed.values)
    num_items = sparse_matrix.shape[0]

    # Initialize an empty similarity matrix
    similarity_matrix = np.zeros((num_items, num_items))

    # Iterate over each row (item) in the transposed matrix by index
    for i in range(num_items):
        for j in range(i, num_items):  # Only compute upper triangle (similarity_matrix is symmetric)
            # Get rows by numeric position using csr_matrix row slicing
            row_i = sparse_matrix.getrow(i)
            row_j = sparse_matrix.getrow(j)

            # Find common indices and adjust to exclude columns with NaN values
            common_indices = np.where(~np.isnan(row_i.toarray()) & ~np.isnan(row_j.toarray()))[1]
    
            if common_indices.size > 0:
                # Get non-missing values for common users
                non_missing_values_i = row_i[:, common_indices].toarray().flatten()
                non_missing_values_j = row_j[:, common_indices].toarray().flatten()

                # Check if both arrays have non-zero length
                if len(non_missing_values_i) > 0 and len(non_missing_values_j) > 0:
                    similarity = cosine_similarity([non_missing_values_i], [non_missing_values_j])[0, 0]
                    similarity_matrix[i, j] = similarity
                    similarity_matrix[j, i] = similarity

    return similarity_matrix

In [53]:
item_similarity = calculate_adjusted_cosine_similarity(item_user_matrix_adjusted)

In [54]:
# Create a DataFrame with item names as both rows and columns
item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix_adjusted.index, columns=item_user_matrix_adjusted.index)

# Now item_item_similarity_df is your item-item similarity matrix with item names as both row and column labels
# Set similarity to 0 for each item to itself
np.fill_diagonal(item_similarity_df.values, -5)
item_similarity_df

BGGId,1,3,5,7,10,11,12,13,15,18,...,304420,306735,308765,312484,314040,316554,317985,318977,328871,329465
BGGId,,,,,,,,,,,,,,,,,,,,,
1,-5.000000,0.341069,0.351651,-0.166453,0.016510,0.131823,0.385674,0.031433,0.029363,0.062802,...,0.300809,0.466279,0.433617,0.300222,0.498090,0.400353,0.430978,0.376563,0.285314,0.031036
3,0.341069,-5.000000,0.345532,-0.082203,0.005425,0.175970,0.502870,0.072141,-0.022112,0.054821,...,0.378477,0.454944,0.385579,0.385751,0.364834,0.402621,0.450432,0.396650,0.409741,0.024460
5,0.351651,0.345532,-5.000000,-0.109793,0.085079,0.206069,0.373398,0.194543,0.076436,0.088442,...,0.185882,0.151204,0.201802,0.199851,0.221072,0.291977,0.277231,0.298402,0.395234,0.075775
7,-0.166453,-0.082203,-0.109793,-5.000000,0.046118,-0.060240,-0.105971,-0.026966,-0.041033,-0.071055,...,-0.348372,-0.360181,-0.211050,-0.352897,-0.450954,-0.289084,-0.357689,-0.251626,-0.268140,0.352918
10,0.016510,0.005425,0.085079,0.046118,-5.000000,0.148711,0.020459,0.145854,0.057443,0.088119,...,-0.251527,-0.342521,-0.278526,-0.188513,-0.231592,-0.290958,-0.246933,-0.101245,-0.270236,0.025802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316554,0.400353,0.402621,0.291977,-0.289084,-0.290958,-0.104215,0.361983,-0.255064,0.026246,-0.080059,...,0.635163,0.685257,0.709343,0.773634,0.793695,-5.000000,0.767165,0.652526,0.721532,0.160422
317985,0.430978,0.450432,0.277231,-0.357689,-0.246933,0.018243,0.431014,-0.245979,-0.013117,-0.061208,...,0.659514,0.672150,0.749071,0.753505,0.799445,0.767165,-5.000000,0.715702,0.739015,0.033233
318977,0.376563,0.396650,0.298402,-0.251626,-0.101245,0.058092,0.384729,-0.127276,-0.050270,0.072220,...,0.574736,0.623932,0.612992,0.677049,0.704459,0.652526,0.715702,-5.000000,0.692889,0.116150


In [55]:
# Convert similarity to distances (inversely proportional to similarities)
distance_matrix = 1 - item_similarity_df

In [56]:
distance_matrix

BGGId,1,3,5,7,10,11,12,13,15,18,...,304420,306735,308765,312484,314040,316554,317985,318977,328871,329465
BGGId,,,,,,,,,,,,,,,,,,,,,
1,6.000000,0.658931,0.648349,1.166453,0.983490,0.868177,0.614326,0.968567,0.970637,0.937198,...,0.699191,0.533721,0.566383,0.699778,0.501910,0.599647,0.569022,0.623437,0.714686,0.968964
3,0.658931,6.000000,0.654468,1.082203,0.994575,0.824030,0.497130,0.927859,1.022112,0.945179,...,0.621523,0.545056,0.614421,0.614249,0.635166,0.597379,0.549568,0.603350,0.590259,0.975540
5,0.648349,0.654468,6.000000,1.109793,0.914921,0.793931,0.626602,0.805457,0.923564,0.911558,...,0.814118,0.848796,0.798198,0.800149,0.778928,0.708023,0.722769,0.701598,0.604766,0.924225
7,1.166453,1.082203,1.109793,6.000000,0.953882,1.060240,1.105971,1.026966,1.041033,1.071055,...,1.348372,1.360181,1.211050,1.352897,1.450954,1.289084,1.357689,1.251626,1.268140,0.647082
10,0.983490,0.994575,0.914921,0.953882,6.000000,0.851289,0.979541,0.854146,0.942557,0.911881,...,1.251527,1.342521,1.278526,1.188513,1.231592,1.290958,1.246933,1.101245,1.270236,0.974198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316554,0.599647,0.597379,0.708023,1.289084,1.290958,1.104215,0.638017,1.255064,0.973754,1.080059,...,0.364837,0.314743,0.290657,0.226366,0.206305,6.000000,0.232835,0.347474,0.278468,0.839578
317985,0.569022,0.549568,0.722769,1.357689,1.246933,0.981757,0.568986,1.245979,1.013117,1.061208,...,0.340486,0.327850,0.250929,0.246495,0.200555,0.232835,6.000000,0.284298,0.260985,0.966767
318977,0.623437,0.603350,0.701598,1.251626,1.101245,0.941908,0.615271,1.127276,1.050270,0.927780,...,0.425264,0.376068,0.387008,0.322951,0.295541,0.347474,0.284298,6.000000,0.307111,0.883850


Calculate nearest neighbor

In [98]:
def find_nearest_neighbors(distance_matrix, BGGId, nearest_neighbors_model):

    # Find the index of the item

    # Find the distance and indices of the nearest neighbors
    distances, neighbor_indices = nearest_neighbors_model.kneighbors([distance_matrix[BGGId]], return_distance=True)

    #print("distance_matrix[BGGId]")
    #print(distance_matrix[BGGId])

    return distances, neighbor_indices

In [ ]:
# def find_nearest_neighbors(distance_matrix, k, BGGId):

#     # Create a NearestNeighbors object
#     nearest_neighbors_model = NearestNeighbors(n_neighbors=k, metric='precomputed')

#     # Fit the model using the item-item similarity matrix
#     nearest_neighbors_model.fit(distance_matrix.values)

#     # Find the index of the item
#     item_index = distance_matrix.index.get_loc(BGGId)

#     # Find the distance and indices of the nearest neighbors
#     distances, neighbor_indices = nearest_neighbors_model.kneighbors([distance_matrix.iloc[item_index]], return_distance=True)
#     print("distance_matrix.iloc[item_index]")
#     print(distance_matrix.iloc[item_index])

#     return distances, neighbor_indices

In [88]:
test = find_nearest_neighbors(distance_matrix, 5, 7)

distance_matrix.iloc[item_index]
BGGId
1         1.166453
3         1.082203
5         1.109793
7         6.000000
10        0.953882
            ...   
316554    1.289084
317985    1.357689
318977    1.251626
328871    1.268140
329465    0.647082
Name: 7, Length: 1167, dtype: float64


In [86]:
def find_rated_neighbors(neighbor_distances, neighbor_indices, rated_items):
    neighbor_indices = neighbor_indices[0]
    neighbor_distances = neighbor_distances[0]
    neighbors = item_user_matrix.iloc[neighbor_indices].index.tolist()
    rated_neighbors = []
    rated_neighbors_distances = []

    for i in range(len(neighbors)):
        if neighbors[i] in rated_items:
            rated_neighbors.append(neighbors[i])
            rated_neighbors_distances.append(neighbor_distances[i])
    
    return rated_neighbors_distances, rated_neighbors

In [99]:
def get_predicted_rating(item, username, distance_matrix, nearest_neighbors_model):
    rated_items = rated_items_train_set[username]
    distances, neighbor_indices = find_nearest_neighbors(distance_matrix, item, nearest_neighbors_model)
    rated_neighbors_distances, rated_neighbors = find_rated_neighbors(distances, neighbor_indices, rated_items)

    if len(rated_neighbors) != 0:
        numerator = 0
        denominator = 0
        for i in range(len(rated_neighbors)):
            neighbor = rated_neighbors[i]
            weight = 1 - rated_neighbors_distances[i] #transform distance into similarity
            user_rating = actual_ratings_train_set[username][neighbor]
            numerator += (weight * user_rating)
            denominator += weight
        predicted_rating_item = numerator / denominator
        return predicted_rating_item
    else:
        average_rating = average_ratings[username]
        return average_rating

In [129]:
def recommend_items(distance_matrix, username, top_N, nearest_neighbors_model):
    not_rated_items = not_rated_items_train_set[username]
    predicted_ratings = []*len(not_rated_items)
    predicted_items = []*len(not_rated_items)

    for item in not_rated_items:
        predicted_rating_item = get_predicted_rating(item, username, distance_matrix, nearest_neighbors_model)
        predicted_ratings.append(predicted_rating_item)
        predicted_items.append(item)  
    
    recommended_items = pd.DataFrame(predicted_ratings, index=predicted_items, columns=['Predicted rating'])
    recommended_items.index.name = 'BGGId'
    sorted_recommendations = recommended_items.sort_values(by=['Predicted rating'], ascending=False)
    top_N_recommendations = sorted_recommendations.head(top_N)
    top_N_list = top_N_recommendations.index.tolist()

    return top_N_list

In [136]:
def calculate_precision(recommended_items, relevant_items):
    true_positives = 0
    for item in recommended_items:
        if item in relevant_items:
            true_positives += 1
    
    return true_positives / len(recommended_items)


In [137]:
def calculate_recall(recommended_items, relevant_items):
    true_positives = 0
    for item in recommended_items:
        if item in relevant_items:
            true_positives += 1
    
    return true_positives / len(relevant_items)

In [138]:
def run_precision_recall_calculation(users, nearest_neighbors_model, top_N):
    results_precision = []
    results_recall = []

    num_users, num_items = user_item_matrix.shape
    for user in users:
        recommended_items = recommend_items(distance_matrix, user, top_N, nearest_neighbors_model)
        relevant_items_user = relevant_items[user]
        precision = calculate_precision(recommended_items, relevant_items_user)
        results_precision.append(precision)
        recall = calculate_recall(recommended_items, relevant_items_user)
        results_recall.append(recall)

    return np.sum(results_precision) / len(users), np.sum(results_recall) / len(users)

In [100]:
def calculate_mean_absolute_error(username, distance_matrix, nearest_neighbors_model):
    actual_ratings = []
    predicted_ratings = []
    rated_items_testset = rated_items_test_set[username]

    for item in rated_items_testset:
        predicted_ratings.append(get_predicted_rating(item, username, distance_matrix, nearest_neighbors_model))
        actual_rating = actual_ratings_test_set[username][item]
        actual_ratings.append(actual_rating)
    
    mae = mean_absolute_error(actual_ratings, predicted_ratings)
    return mae

In [67]:
def get_user_sample(random_seed, size):
    sample = set()
    num_users, num_items = user_item_matrix.shape
    np.random.seed(random_seed)
    while len(sample) < 1000:
        random_user_index = np.random.randint(0, num_users)
        random_user = user_item_matrix.index[random_user_index]
        sample.add(random_user)
    return sample
    

In [154]:
def run_mean_absolute_error_calculation(users, nearest_neighbors_model):
    results = []
    num_users, num_items = user_item_matrix.shape
    for user in users:
        mae = calculate_mean_absolute_error(user, distance_matrix, nearest_neighbors_model)
        results.append(mae)
    return np.sum(results) / len(users)


In [109]:
# Create a NearestNeighbors object
nearest_neighbors_model = NearestNeighbors(n_neighbors=5, metric='precomputed')

# Fit the model using the item-item similarity matrix
nearest_neighbors_model.fit(distance_matrix.values)

NearestNeighbors(metric='precomputed')

In [104]:
mae_k5 = run_mean_absolute_error_calculation(user_sample, nearest_neighbors_model)


Mean absolute error with k =  and num_users = 1000 : 1.0217391304347827


In [115]:
nearest_neighbors_model = NearestNeighbors(n_neighbors=5, metric='precomputed')
nearest_neighbors_model.fit(distance_matrix.values)
mae_k5 = run_mean_absolute_error_calculation(user_sample, nearest_neighbors_model)

NearestNeighbors(metric='precomputed')

In [155]:
user_sample = get_user_sample(0, 1000)
k_values = [5,10,15,20,25,30,35,40,45,50,100]
results_mae_user_sample_0 = {}
for k in k_values:
    nearest_neighbors_model = NearestNeighbors(n_neighbors=k, metric='precomputed')
    nearest_neighbors_model.fit(distance_matrix.values)
    result = run_mean_absolute_error_calculation(user_sample, nearest_neighbors_model)
    print("Mean absolute error with k = ",k ,"and num_users =", len(user_sample),":", result)
    results_mae_user_sample_0[k] = result

Mean absolute error with k =  2 and num_users = 1000 : 0.9700743308699096
Mean absolute error with k =  3 and num_users = 1000 : 0.9750052257031734


In [ ]:
user_sample = get_user_sample(1, 1000)
k_values = [5,10,15,20,25,30,35,40,45,50,100]
results_mae_user_sample_1 = {}
for k in k_values:
    nearest_neighbors_model = NearestNeighbors(n_neighbors=k, metric='precomputed')
    nearest_neighbors_model.fit(distance_matrix.values)
    result = run_mean_absolute_error_calculation(user_sample, nearest_neighbors_model)
    print("Mean absolute error with k = ",k ,"and num_users =", len(user_sample),":", result)
    results_mae_user_sample_1[k] = result

In [ ]:
user_sample = get_user_sample(2, 1000)
k_values = [5,10,15,20,25,30,35,40,45,50,100]
results_mae_user_sample_2 = {}
for k in k_values:
    nearest_neighbors_model = NearestNeighbors(n_neighbors=k, metric='precomputed')
    nearest_neighbors_model.fit(distance_matrix.values)
    result = run_mean_absolute_error_calculation(user_sample, nearest_neighbors_model)
    print("Mean absolute error with k = ",k ,"and num_users =", len(user_sample),":", result)
    results_mae_user_sample_2[k] = result

In [139]:
precision_k5, recall_k5 = run_precision_recall_calculation(user_sample, nearest_neighbors_model, 10)

In [141]:
precision_k5

0.0782

In [142]:
recall_k5

0.030555537524588765

In [ ]:
# update model
# mae_k10 = run_mean_absolute_error_calculation(10, user_sample)

In [84]:
test_mae = calculate_mean_absolute_error("-=Yod@=-", 20, distance_matrix)
test_mae

1.0239391745226554

In [116]:
test_recommended_items, test_rec_df = recommend_items(distance_matrix, "-=Yod@=-", 10, nearest_neighbors_model)
test_recommended_items

[132531,
 189932,
 192135,
 162886,
 274637,
 161936,
 270844,
 268864,
 266507,
 262712]

In [127]:
test_recommended_items, test_rec_df = recommend_items(distance_matrix, "-=Yod@=-", 10, nearest_neighbors_model)
test_recommended_items

[132531,
 189932,
 192135,
 162886,
 274637,
 161936,
 270844,
 268864,
 266507,
 262712]

In [108]:
test_recommended_items, test_rec_df = recommend_items(distance_matrix, "-=Yod@=-", 30, nearest_neighbors_model)
test_recommended_items

[192135,
 166669,
 204472,
 159675,
 229853,
 271324,
 176734,
 241451,
 193738,
 177736,
 28720,
 256960,
 274637,
 191189,
 119506,
 276025,
 251247,
 249259,
 296151,
 218417,
 9216,
 224710,
 266810,
 317985,
 189932,
 316554,
 314040,
 312484,
 224517,
 308765]

In [74]:
relevant_items

,BGGId,Rating,Rating_avg,Adjusted_Rating,Username,Name
0,2381,7.5,7.287829,0.212171,-=Yod@=-,Scattergories
1,257501,8.0,7.287829,0.712171,-=Yod@=-,KeyForge: Call of the Archons
2,11,7.0,7.287829,-0.287829,-=Yod@=-,Bohnanza
3,139030,7.5,7.287829,0.212171,-=Yod@=-,Mascarade
4,24480,7.0,7.287829,-0.287829,-=Yod@=-,The Pillars of the Earth
...,...,...,...,...,...,...
1204355,161533,8.0,7.393506,0.606494,zzzzzane,Lisboa
1204356,150,6.0,7.393506,-1.393506,zzzzzane,PitchCar
1204357,320,8.0,7.393506,0.606494,zzzzzane,Scrabble
1204358,521,10.0,7.393506,2.606494,zzzzzane,Crokinole


In [114]:
# Username
# leffe dubbel            1081
# Hessu68                 1020
# oldgoat3769967          1002
# TomVasel                 996
# Pandorzecza              953
#                         ... 
# RoN_Rancor               100
# generationX              100
# Nathan G                 100
# starvinmartian_games     100
# chsbellboy               100

#users: -Johnny-	-mIDE-	-toni-	...Hammer	
test_recommended_items_2, test_recommended_items_2_df = recommend_items(distance_matrix, "leffe dubbel", 10, nearest_neighbors_model)
test_relevant_items_2 = relevant_items["leffe dubbel"]
test_precision_2 = calculate_precision(test_recommended_items_2, test_relevant_items_2)
test_precision_2

9625
25021
188
79828
54138
39351
237182
84419
163068
91
antal true positives:  10


1.0

In [77]:
len(test_relevant_items_2)

154

In [78]:
test_recommended_items_2_df

,Predicted rating
BGGId,
9625,8.072541
25021,8.058141
188,8.018426
79828,8.002364
54138,7.998041
39351,7.946282
237182,7.941399
84419,7.912132
163068,7.892771


Nedan skriver vi funktion som visar resultat av rekomendation i form av en sorts lista baserat på recommender funktionen

Nedan här skriver vi evalueringsfunktioner av modellen och utskrift 